Tahap-tahap berikut dilakukan mulai dari impor library, membaca dataset, mengeksplorasi data, memeriksa informasi, hingga pembersihan.

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, Concatenate, Dense, Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Concatenate

In [ ]:
articlesDF = pd.read_csv('/content/AdvertisemetRecommenderSystem-with-ArticleDatasets/Dataset/Articles.csv')
usersDF = pd.read_csv('/content/AdvertisemetRecommenderSystem-with-ArticleDatasets/Dataset/Users.csv')

In [ ]:
articlesDF.shape

(6877, 3)

In [ ]:
usersDF.head()

,Daily Time Spent on Site,Age,Area Income,Daily Internet Usage,Ad Topic Line,City,Male,Country,Timestamp,Clicked on Ad
0,68.95,35,61833.90,256.09,Cloned 5thgeneration orchestration,Wrightburgh,0,Tunisia,2016-03-27 00:53:11,0
1,80.23,31,68441.85,193.77,Monitored national standardization,West Jodi,1,Nauru,2016-04-04 01:39:02,0
2,69.47,26,59785.94,236.50,Organic bottom-line service-desk,Davidton,0,San Marino,2016-03-13 20:35:42,0
3,74.15,29,54806.18,245.89,Triple-buffered reciprocal time-frame,West Terrifurt,1,Italy,2016-01-10 02:31:19,0
4,68.37,35,73889.99,225.58,Robust logistical utilization,South Manuel,0,Iceland,2016-06-03 03:36:18,0


In [ ]:
articlesDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6877 entries, 0 to 6876
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   category  6877 non-null   object
 1   title     6877 non-null   object
 2   body      6872 non-null   object
dtypes: object(3)
memory usage: 161.3+ KB


In [ ]:
usersDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 10 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Daily Time Spent on Site  1000 non-null   float64
 1   Age                       1000 non-null   int64  
 2   Area Income               1000 non-null   float64
 3   Daily Internet Usage      1000 non-null   float64
 4   Ad Topic Line             1000 non-null   object 
 5   City                      1000 non-null   object 
 6   Male                      1000 non-null   int64  
 7   Country                   1000 non-null   object 
 8   Timestamp                 1000 non-null   object 
 9   Clicked on Ad             1000 non-null   int64  
dtypes: float64(3), int64(3), object(4)
memory usage: 78.2+ KB


In [ ]:
articlesDF = articlesDF.drop(columns = ['body'], axis=1)

In [ ]:
articlesDF.head()

,category,title
0,ARTS & CULTURE,Modeling Agencies Enabled Sexual Predators For...
1,ARTS & CULTURE,Actor Jeff Hiller Talks “Bright Colors And Bol...
2,ARTS & CULTURE,New Yorker Cover Puts Trump 'In The Hole' Afte...
3,ARTS & CULTURE,Man Surprises Girlfriend By Drawing Them In Di...
4,ARTS & CULTURE,This Artist Gives Renaissance-Style Sculptures...


In [ ]:
usersDF = usersDF.drop(columns = ['Daily Internet Usage', 'Ad Topic Line', 'City', 'Male', 'Country',
       'Timestamp', 'Clicked on Ad'], axis=1)

Menghilangkan beberapa kolom dari DataFrame `usersDF` dilakukan dengan menggunakan fungsi `.drop()`. Parameter `columns` digunakan untuk menentukan nama-nama kolom yang ingin dihapus, yaitu: 'Daily Internet Usage', 'Ad Topic Line', 'City', 'Male', 'Country', 'Timestamp', dan 'Clicked on Ad'.

In [ ]:
usersDF.head()

,Daily Time Spent on Site,Age,Area Income
0,68.95,35,61833.90
1,80.23,31,68441.85
2,69.47,26,59785.94
3,74.15,29,54806.18
4,68.37,35,73889.99


In [ ]:
articlesDF['category'].value_counts()

,count
category,
ARTS & CULTURE,1002
BUSINESS,501
ENTERTAINMENT,501
ENVIRONMENT,501
POLITICS,501
RELIGION,501
SPORTS,501
TECH,501
WOMEN,501


In [ ]:
articlesDF = articlesDF.groupby('category', group_keys=False).apply(lambda x: x.sample(min(len(x), 300)))

<ipython-input-13-9cfe1e2063f0>:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  articlesDF = articlesDF.groupby('category', group_keys=False).apply(lambda x: x.sample(min(len(x), 300)))


Memastikan bahwa jumlah baris untuk setiap kategori dalam kolom 'category' tidak melebihi 300.

In [ ]:
articlesDF['category'].value_counts()

,count
category,
ARTS & CULTURE,300
BUSINESS,300
COMEDY,300
CRIME,300
EDUCATION,300
ENTERTAINMENT,300
ENVIRONMENT,300
MEDIA,300
POLITICS,300


In [ ]:
usersDF['userId'] = [i for i in range(1, len(usersDF) + 1)]

Menyisipkan kolom `userID` guna mempermudah proses identifikasi atau pelacakan data.

In [ ]:
usersDF

,Daily Time Spent on Site,Age,Area Income,userId
0,68.95,35,61833.90,1
1,80.23,31,68441.85,2
2,69.47,26,59785.94,3
3,74.15,29,54806.18,4
4,68.37,35,73889.99,5
...,...,...,...,...
995,72.97,30,71384.57,996
996,51.30,45,67782.17,997
997,51.63,51,42415.72,998
998,55.55,19,41920.79,999


In [ ]:
articlesDF['articleId'] = ['article_' + str(i) for i in range(1, len(articlesDF) + 1)]

Menambahkan kolom `articleID` untuk mempermudah proses pelacakan.

In [ ]:
articlesDF

,category,title,articleId
390,ARTS & CULTURE,"To Protest Trump's Travel Ban, Museum Temporar...",article_1
458,ARTS & CULTURE,Artist Turns Donald Trump’s Most Controversial...,article_2
750,ARTS & CULTURE,You've Got Me Under Your Spell,article_3
936,ARTS & CULTURE,Does Ballet Have a Color?,article_4
109,ARTS & CULTURE,12 Of The Most Fly Photos Taken By Drones In 2017,article_5
...,...,...,...
6507,WOMEN,MSU Athletic Director Mark Hollis Resigns In W...,article_4196
6489,WOMEN,Arizona Department Of Corrections Changes Mens...,article_4197
6528,WOMEN,A TV Series About The #MeToo Movement Is Comin...,article_4198
6408,WOMEN,Southwest's Tammie Jo Shults Joins The Legacy ...,article_4199


In [ ]:
dummy_data = []

In [ ]:
for user in usersDF["userId"]:
    chosen_articles = np.random.choice(articlesDF["articleId"], 30, replace=False)
    for article in chosen_articles:
        dummy_data.append({"userId": user, "articleId": article})

Membuat data dummy yang merepresentasikan interaksi pengguna dengan dataset iklan (artikel).

In [ ]:
user_article_df = pd.DataFrame(dummy_data)

In [ ]:
unique_counts = user_article_df.value_counts().unique()

print("Unique counts of userId and articleId combinations:", unique_counts)

if len(unique_counts) == 1 and unique_counts[0] == 1:
    print("Validation passed: No duplicate user-article combinations.")
else:
    print("Validation failed: Duplicate combinations found!")

Unique counts of userId and articleId combinations: [1]
Validation passed: No duplicate user-article combinations.


In [ ]:
user_article_df.head(12)

,userId,articleId
0,1,article_1889
1,1,article_2306
2,1,article_2888
3,1,article_1049
4,1,article_815
5,1,article_3213
6,1,article_488
7,1,article_3664
8,1,article_2090
9,1,article_2139


In [ ]:
articlesDF.head(12)

,category,title,articleId
390,ARTS & CULTURE,"To Protest Trump's Travel Ban, Museum Temporar...",article_1
458,ARTS & CULTURE,Artist Turns Donald Trump’s Most Controversial...,article_2
750,ARTS & CULTURE,You've Got Me Under Your Spell,article_3
936,ARTS & CULTURE,Does Ballet Have a Color?,article_4
109,ARTS & CULTURE,12 Of The Most Fly Photos Taken By Drones In 2017,article_5
926,ARTS & CULTURE,The Mayhem Of 'Murphy',article_6
902,ARTS & CULTURE,Why We Must Value Languages,article_7
852,ARTS & CULTURE,Blinds and Bells: Haegue Yang's Retrospective ...,article_8
581,ARTS & CULTURE,Post-Internet to Proto-Internet: How Has the D...,article_9
511,ARTS & CULTURE,The First,article_10


In [ ]:
usersDF.head(12)

,Daily Time Spent on Site,Age,Area Income,userId
0,68.95,35,61833.90,1
1,80.23,31,68441.85,2
2,69.47,26,59785.94,3
3,74.15,29,54806.18,4
4,68.37,35,73889.99,5
5,59.99,23,59761.56,6
6,88.91,33,53852.85,7
7,66.00,48,24593.33,8
8,74.53,30,68862.00,9
9,69.88,20,55642.32,10


In [ ]:
interactions_df = user_article_df.merge(articlesDF, on='articleId')
interactions_df.head(5)

,userId,articleId,category,title
0,1,article_1889,ENVIRONMENT,UN Climate Talks May Only Bring Moderate Progr...
1,1,article_2306,MEDIA,Tucker Carlson Turns On Trump: 'Imagine If Bar...
2,1,article_2888,RELIGION,When Patriotism Becomes Idolatry
3,1,article_1049,CRIME,Cops Explain How To Pull Off A Good Halloween ...
4,1,article_815,COMEDY,'Late Night' Writer's Breathless Royal Wedding...


In [ ]:
user_category_matrix = interactions_df.groupby(["userId", "category"]).size().unstack(fill_value=0)

In [ ]:
user_category_matrix

category,ARTS & CULTURE,BUSINESS,COMEDY,CRIME,EDUCATION,ENTERTAINMENT,ENVIRONMENT,MEDIA,POLITICS,RELIGION,SCIENCE,SPORTS,TECH,WOMEN
userId,,,,,,,,,,,,,,
1,3,2,1,3,1,2,3,4,2,1,4,0,3,1
2,4,2,1,3,3,1,4,0,2,1,2,0,5,2
3,2,1,2,1,4,2,2,3,4,2,0,2,2,3
4,3,5,1,2,1,4,0,2,1,2,3,3,1,2
5,2,0,1,0,2,2,4,2,5,1,3,3,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,0,2,1,0,0,4,2,1,3,1,2,3,3,8
997,3,2,1,0,3,0,3,1,4,1,4,3,4,1
998,4,0,1,1,2,2,3,0,2,1,2,1,6,5


In [ ]:
num_categories = len(user_category_matrix.columns)

user_input = Input(shape=(num_categories,), name='user_input')
category_input = Input(shape=(num_categories,), name='category_input')

merged = Concatenate()([user_input, category_input])

hidden_1 = Dense(64, activation='relu')(merged)
hidden_2 = Dense(32, activation='relu')(hidden_1)
output = Dense(1, activation='sigmoid', name='output')(hidden_2)

model = Model(inputs=[user_input, category_input], outputs=output)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ user_input (InputLayer)   │ (None, 14)             │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ category_input            │ (None, 14)             │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate (Concatenate) │ (None, 28)             │              0 │ user_input[0][0],      │
│                           │                        │                │ category_input[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 64)             │          1,856 │ concatenate[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_1 (Dense)           │ (None, 32)             │          2,080 │ dense[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ output (Dense)            │ (None, 1)              │             33 │ dense_1[0][0]          │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 3,969 (15.50 KB)

 Trainable params: 3,969 (15.50 KB)

 Non-trainable params: 0 (0.00 B)

Model neural network ini digunakan untuk memprediksi output biner (0 atau 1). Model memiliki dua input, yaitu `user_input` dan `category_input`, yang masing-masing memiliki panjang sama dengan jumlah kategori pada `user_category_matrix`. Kedua input tersebut digabungkan melalui layer Concatenate. Selanjutnya, terdapat dua lapisan tersembunyi dengan 64 dan 32 neuron yang menggunakan fungsi aktivasi ReLU untuk pemrosesan lebih lanjut. Output model berupa satu neuron dengan aktivasi sigmoid, yang menghasilkan prediksi biner. Model ini dikompilasi menggunakan optimizer Adam dan fungsi loss `binary_crossentropy` khusus untuk klasifikasi biner.

In [ ]:
user_preferences = user_category_matrix.values

articles = pd.get_dummies(articlesDF['category'])

user_ids = interactions_df['userId'].unique()
categories = articles.columns

num_categories = len(categories)
X_user = []
X_category = []
y = []

for user in user_ids:
    for category in categories:
        X_user.append(user_category_matrix.loc[user].values if user in user_category_matrix.index else [0] * num_categories)

        category_vector = np.zeros(num_categories)
        category_index = list(categories).index(category)
        category_vector[category_index] = 1
        X_category.append(category_vector)

        interacted = interactions_df[
            (interactions_df['userId'] == user) & (interactions_df['category'] == category)
        ]
        y.append(1 if not interacted.empty else 0)



X_user = np.array(X_user)
X_category = np.array(X_category)
y = np.array(y)

Mempersiapkan data untuk melatih model yang memprediksi interaksi antara pengguna dan kategori artikel. Untuk setiap kombinasi pengguna (`userId`) dan kategori artikel, dibuat dua input: `X_user`, yaitu vektor preferensi pengguna terhadap kategori, dan `X_category`, berupa vektor one-hot encoding kategori artikel tersebut. Target `y` menunjukkan ada atau tidaknya interaksi pengguna dengan artikel di kategori tersebut (1 jika berinteraksi, 0 jika tidak). Semua data untuk tiap pasangan pengguna dan kategori dikumpulkan ke dalam array NumPy (`X_user`, `X_category`, dan `y`) yang siap dipakai untuk pelatihan model prediksi interaksi.

In [ ]:
model.fit([X_user, X_category], y, epochs=5, batch_size=32)

Epoch 1/5
438/438 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8888 - loss: 0.3464
Epoch 2/5
438/438 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.8905 - loss: 0.3347
Epoch 3/5
438/438 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8981 - loss: 0.3073
Epoch 4/5
438/438 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8956 - loss: 0.2955
Epoch 5/5
438/438 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8925 - loss: 0.2786


In [ ]:
new_user_articles = ['article_1021', 'article_2364', 'article_271', 'article_1741', 'article_3485']

In [ ]:
new_user_vector = np.zeros(num_categories)

for article_id in new_user_articles:
    article_category = articlesDF.loc[articlesDF['articleId'] == article_id, 'category'].values[0]
    category_index = list(categories).index(article_category)
    new_user_vector[category_index] += 1

new_user_vector = new_user_vector / len(new_user_articles)
print("New user preference vector:", new_user_vector)


New user preference vector: [0.2 0.  0.  0.2 0.  0.2 0.  0.2 0.  0.  0.  0.2 0.  0. ]


In [ ]:
preferred_categories = [categories[i] for i in range(num_categories) if new_user_vector[i] > 0]

potential_recommendations = articlesDF[articlesDF['category'].isin(preferred_categories)]
print(potential_recommendations)


            category                                              title  \
390   ARTS & CULTURE  To Protest Trump's Travel Ban, Museum Temporar...   
458   ARTS & CULTURE  Artist Turns Donald Trump’s Most Controversial...   
750   ARTS & CULTURE                     You've Got Me Under Your Spell   
936   ARTS & CULTURE                          Does Ballet Have a Color?   
109   ARTS & CULTURE  12 Of The Most Fly Photos Taken By Drones In 2017   
...              ...                                                ...   
5628          SPORTS  14 Michigan State Reps Reportedly Heard About ...   
5799          SPORTS   'How Many Reds Does It Take To Catch A Pop Fly?'   
5530          SPORTS  Shaun White Is Damn Near Perfect In His Winter...   
5477          SPORTS  French Skier Booted From Olympics After Angry ...   
5756          SPORTS  Conor McGregor Calls Floyd Mayweather's Strip ...   

         articleId  
390      article_1  
458      article_2  
750      article_3  
936      articl

Di bagian ini, dibuat sebuah vektor kosong dengan panjang sesuai jumlah kategori. Kemudian, untuk setiap artikel yang dibaca oleh pengguna baru (`new_user_articles`), kategori artikel tersebut diidentifikasi, dan nilai pada posisi kategori yang bersangkutan di dalam vektor preferensi diperbarui. Vektor ini kemudian dinormalisasi dengan membagi setiap nilainya dengan total artikel yang telah dibaca. Selanjutnya, kategori dengan nilai lebih dari nol diambil dari vektor preferensi dan digunakan untuk menyaring artikel-artikel yang masuk dalam kategori tersebut sebagai rekomendasi potensial bagi pengguna baru.

In [ ]:
recommendations = []

for _, article in potential_recommendations.iterrows():
    article_category_vector = np.zeros(num_categories)
    article_index = list(categories).index(article['category'])
    article_category_vector[article_index] = 1

    probability = model.predict([np.array([new_user_vector]), np.array([article_category_vector])], verbose = 0)[0][0]

    recommendations.append((article['articleId'], article['title'], probability))

In [ ]:
recommendations = sorted(recommendations, key=lambda x: x[2], reverse=True)

**Top N (5) Recommendation For Content-Based Filtering**

In [ ]:
top_n = 5
print("Top recommendations:")
for article_id, title, prob in recommendations[:top_n]:
    print(f"Article: {title} (ID: {article_id}, PROB: {prob:.2f})")

Top recommendations:
Article: Anderson Cooper Shreds Trump: ‘He Went To Play Golf While They Held Funerals’ (ID: article_2101, PROB: 0.57)
Article: How Many Times Can Wolf Blitzer Avoid Saying 'Shithole'? (ID: article_2102, PROB: 0.57)
Article: Parents Of Slain DNC Staffer Seth Rich Sue Fox News Over Fake News Story (ID: article_2103, PROB: 0.57)
Article: Media Euphemisms For 'Racist' Are Stupidly Tinged (ID: article_2104, PROB: 0.57)
Article: Ted Cruz Makes It Too Easy To Point Out The Hypocrisy Of His Latest Campaign Ad (ID: article_2105, PROB: 0.57)


Membuat rekomendasi artikel untuk pengguna baru dengan menggunakan prediksi dari model. Setiap artikel dalam `potential_recommendations` diproses secara individual untuk membentuk vektor kategori dalam format one-hot encoding.

In [ ]:
scaler = MinMaxScaler()
usersDF[['Daily Time Spent on Site', 'Age', 'Area Income']] = scaler.fit_transform(
    usersDF[['Daily Time Spent on Site', 'Age', 'Area Income']]
)

article_ids = interactions_df['articleId'].unique()
article_to_idx = {article: idx for idx, article in enumerate(article_ids)}
interactions_df['article_idx'] = interactions_df['articleId'].map(article_to_idx)

num_users = len(usersDF)
num_articles = len(article_ids)
interactions_matrix = np.zeros((num_users, num_articles))

for _, row in interactions_df.iterrows():
    interactions_matrix[row['userId'] - 1, row['article_idx']] = 1

In [ ]:
embedding_dim = 50

user_input = Input(shape=(3,), name='user_features')  # [Age, Income, Time Spent]

user_id_input = Input(shape=(1,), name='user_id')
user_embedding = Embedding(input_dim=num_users, output_dim=embedding_dim, name='user_embedding')(user_id_input)
user_embedding = Flatten()(user_embedding)

article_input = Input(shape=(1,), name='article_id')
article_embedding = Embedding(input_dim=num_articles, output_dim=embedding_dim, name='article_embedding')(article_input)
article_embedding = Flatten()(article_embedding)

merged = Concatenate()([user_embedding, article_embedding, user_input])

hidden = Dense(32, activation='relu')(merged)
hidden = Dense(16, activation='relu')(hidden)
output = Dense(1, activation='sigmoid', name='output')(hidden)

model = Model(inputs=[user_id_input, article_input, user_input], outputs=output)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ user_id (InputLayer)      │ (None, 1)              │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ article_id (InputLayer)   │ (None, 1)              │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ user_embedding            │ (None, 1, 50)          │         50,000 │ user_id[0][0]          │
│ (Embedding)               │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ article_embedding         │ (None, 1, 50)          │        209,900 │ article_id[0][0]       │
│ (Embedding)               │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten (Flatten)         │ (None, 50)             │              0 │ user_embedding[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten_1 (Flatten)       │ (None, 50)             │              0 │ article_embedding[0][… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ user_features             │ (None, 3)              │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_1             │ (None, 103)            │              0 │ flatten[0][0],         │
│ (Concatenate)             │                        │                │ flatten_1[0][0],       │
│                           │                        │                │ user_features[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_2 (Dense)           │ (None, 32)             │          3,328 │ concatenate_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_3 (Dense)           │ (None, 16)             │            528 │ dense_2[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ output (Dense)            │ (None, 1)              │             17 │ dense_3[0][0]          │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 263,773 (1.01 MB)

 Trainable params: 263,773 (1.01 MB)

 Non-trainable params: 0 (0.00 B)

Model neural network dibangun untuk memprediksi interaksi antara pengguna dan artikel dengan memadukan embedding dan fitur tambahan pengguna. Data pengguna (`usersDF`) dinormalisasi pada fitur seperti durasi waktu, usia, dan pendapatan menggunakan MinMaxScaler. Setiap artikel diberikan indeks unik, dan interaksi antara pengguna dan artikel direpresentasikan dalam bentuk matriks biner (`interactions_matrix`). Model menerima tiga input: ID pengguna (`user_id_input`), ID artikel (`article_input`), dan fitur tambahan pengguna (`user_features`). Embedding dipakai untuk merepresentasikan pengguna dan artikel dalam ruang berdimensi rendah (50 dimensi). Hasil embedding diratakan lalu digabungkan dengan fitur tambahan pengguna melalui layer Concatenate. Data kemudian diproses oleh dua lapisan tersembunyi dengan 32 dan 16 neuron sebelum menghasilkan output probabilitas interaksi lewat layer akhir dengan aktivasi sigmoid. Model ini dikompilasi menggunakan optimizer Adam dan fungsi loss `binary_crossentropy` untuk klasifikasi biner.

In [ ]:
positive_samples = interactions_df[['userId', 'article_idx']]
positive_samples['label'] = 1

all_user_ids = np.arange(num_users)
all_article_ids = np.arange(num_articles)

all_pairs = pd.MultiIndex.from_product([all_user_ids, all_article_ids], names=["userId", "article_idx"]).to_frame(index=False)

negative_samples = all_pairs.merge(positive_samples, on=['userId', 'article_idx'], how='left', indicator=True)
negative_samples = negative_samples[negative_samples['_merge'] == 'left_only'].drop(columns=['_merge'])
negative_samples['label'] = 0

training_data = pd.concat([positive_samples, negative_samples.sample(len(positive_samples))])  # Balance classes

<ipython-input-40-e8512aa057d4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  positive_samples['label'] = 1


In [ ]:
X_user_id = training_data['userId'].values -1
X_article_id = training_data['article_idx'].values
X_user_features = usersDF[['Age', 'Area Income', 'Daily Time Spent on Site']].values[X_user_id]
y = training_data['label'].values

In [ ]:
model.fit(
    [X_user_id, X_article_id, X_user_features],
    y,
    epochs=5,
    batch_size=32
)

Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 11s 4ms/step - accuracy: 0.4943 - loss: 0.6938
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 13s 6ms/step - accuracy: 0.5528 - loss: 0.6868
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 19s 5ms/step - accuracy: 0.6280 - loss: 0.6490
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.7066 - loss: 0.5723
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 11s 6ms/step - accuracy: 0.7689 - loss: 0.4889


Model dilatih menggunakan data positif dan negatif untuk memprediksi interaksi antara pengguna dan artikel. Data positif (`positive_samples`) berisi pasangan pengguna-artikel dari `interactions_df` dengan label 1. Data negatif (`negative_samples`) dibuat dengan terlebih dahulu menghasilkan semua kemungkinan pasangan pengguna-artikel, kemudian menghapus pasangan yang sudah ada di data positif. Pasangan negatif ini diberi label 0, dan jumlahnya disesuaikan agar seimbang dengan data positif. Fitur input meliputi ID pengguna (`X_user_id`), ID artikel (`X_article_id`), dan fitur tambahan pengguna (`X_user_features`) yang diambil dari DataFrame `usersDF` berdasarkan ID pengguna. Target (`y`) merupakan label interaksi biner (1 atau 0). Model kemudian dilatih selama 5 epoch dengan batch size 32 menggunakan fungsi `model.fit()`.

**Top N (5) Recommendations Collaborative Filtering**

In [ ]:
def recommend_articles_for_new_user(user_features, interacted_articles, top_n=5):
    normalized_features = scaler.transform([user_features])

    user_input = np.array([normalized_features[0]] * num_articles)
    article_input = np.arange(num_articles)

    probabilities = model.predict([np.zeros(len(article_input)), article_input, user_input])

    excluded_articles = set(interacted_articles)
    article_probabilities = [
        (article_ids[idx], prob)
        for idx, prob in enumerate(probabilities.flatten())
        if article_ids[idx] not in excluded_articles
    ]

    article_probabilities.sort(key=lambda x: x[1], reverse=True)

    recommended_articles = article_probabilities[:top_n]

    recommendations_with_titles = [
        {
            "article_id": article_id,
            "title": articlesDF.loc[articlesDF["articleId"] == article_id, "title"].values[0],
            "probability": prob,
        }
        for article_id, prob in recommended_articles
    ]

    return recommendations_with_titles


new_user_features = [20, 50000, 70]
interacted_articles = ['article_251', 'article_999', 'article_78']

recommendations = recommend_articles_for_new_user(new_user_features, interacted_articles, top_n=5)

print("Recommended articles for the new user:")
for recommendation in recommendations:
    print(f"Article ID: {recommendation['article_id']}, Title: {recommendation['title']}")

 24/132 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


132/132 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
Recommended articles for the new user:
Article ID: article_1889, Title: UN Climate Talks May Only Bring Moderate Progress As Rich Nations Worry About Economy
Article ID: article_2306, Title: Tucker Carlson Turns On Trump: 'Imagine If Barack Obama Had Said That'
Article ID: article_2888, Title: When Patriotism Becomes Idolatry
Article ID: article_1049, Title: Cops Explain How To Pull Off A Good Halloween Prank Without Getting Arrested
Article ID: article_815, Title: 'Late Night' Writer's Breathless Royal Wedding Recap Is The Only One You Need


Memberikan rekomendasi artikel kepada pengguna baru berdasarkan fitur mereka dan riwayat artikel yang telah dibaca. Pertama, fitur pengguna (`user_features`) dinormalisasi menggunakan scaler yang telah dipelajari sebelumnya. Seluruh artikel yang tersedia dijadikan kandidat dengan membuat input pengguna (`user_input`) berupa penggandaan fitur yang sudah dinormalisasi, serta input artikel (`article_input`) berupa indeks semua artikel. Model kemudian memprediksi probabilitas interaksi untuk setiap artikel. Artikel yang sudah pernah dilihat oleh pengguna (`interacted_articles`) dikeluarkan dari daftar rekomendasi. Artikel yang tersisa disusun berdasarkan probabilitas interaksi dari yang tertinggi ke terendah, lalu sejumlah `top_n` artikel terbaik dikembalikan sebagai rekomendasi. Hasil rekomendasi ditampilkan dengan mencetak ID artikel yang dipilih.